In [40]:
import requests #请求网页
from typing import Dict #类型提示
from PIL import Image,ImageTk #图片显示和保存
import io #配合PIL使用
import tkinter as tk #以便图片显示和交互式保存
import os  #创建图片保存的文件夹

##### 把字符串转化为字典 需要每对键值间以'\n'间隔,键值内以':'间隔 #####

In [3]:
def str_to_dict(s:str)->Dict:
    return {line.split(':',1)[0].strip():line.split(':',1)[1].strip() for line in s.split('\n') if line!=''}

##### 创建保存图片的文件夹 #####

In [41]:
dir_path='./表情包'
if not os.path.isdir(dir_path):
    os.makedirs(dir_path)

## 根据关键词发送请求，解析响应对象，得到图片集的路径/网址 ##

In [4]:
#准备发送请求的参数
key='狗狗'
cur_page=1
page_size=100
search_headers_str='''Accept:application/json, text/plain, */*
Accept-Encoding:gzip, deflate, br
Accept-Language:zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6
Origin:https://www.doutub.com
Referer:https://www.doutub.com/
Sec-Ch-Ua:"Microsoft Edge";v="119", "Chromium";v="119", "Not?A_Brand";v="24"
Sec-Ch-Ua-Mobile:?0
Sec-Ch-Ua-Platform:"Windows"
Sec-Fetch-Dest:empty
Sec-Fetch-Mode:cors
Sec-Fetch-Site:same-site
User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0
'''
search_params_str=f'''keyword: {key}
curPage: {cur_page}
pageSize: {page_size}
'''
search_url='https://api.doutub.com/api/bq/search'
search_headers=str_to_dict(search_headers_str)
search_params=str_to_dict(search_params_str)

In [6]:
search_resp=requests.get(url=search_url,headers=search_headers,params=search_params)

#提取信息
search_resp_content=search_resp.content
search_resp_dict=search_resp.json()
search_resp_data=search_resp_dict['data']

In [8]:
count=search_resp_data['count'] #图片集的数量
imgs_name=[img['imgName'] for img in search_resp_data['rows']]
imgs_path=[img['path'] for img in search_resp_data['rows']] #网址

## 对于图片集的一个图片的网址进行请求并解析 ##

In [9]:
img_headers_str='''
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Encoding: gzip, deflate, br
Accept-Language: zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6
Cache-Control: max-age=0
Connection: keep-alive
Cookie: Hm_lvt_0e35b200a6045f9dd8f8887cd4626da2=1701179179; Hm_lpvt_0e35b200a6045f9dd8f8887cd4626da2=1701179762
Host: qn.doutub.com
If-Modified-Since: Wed, 24 Jul 2019 14:27:31 GMT
If-None-Match: "FoaI7uga9nS0wUQm0LCqu3fxtNHH"
Referer: https://www.doutub.com/
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-site
Sec-Fetch-User: ?1
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0
sec-ch-ua: "Microsoft Edge";v="119", "Chromium";v="119", "Not?A_Brand";v="24"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Windows"'''
img_headers=str_to_dict(img_headers_str)

#一个例子 选取第1张图片
img_name=imgs_name[0]
img_url=imgs_path[0]

img_resp=requests.get(url=img_url,headers=img_headers)

In [52]:
img_type=img_resp.headers['Content-Type'].split('/')[1].strip() #解析响应对象的类型 very important skill !!!

##### 用PIL进行图片读取 #####

In [58]:
img_bytes=io.BytesIO(img_resp.content)
image=Image.open(img_bytes)
image.show()

##### 用tkinter进行交互式显示/保存图片 #####

In [63]:
root=tk.Tk()
tk_image=ImageTk.PhotoImage(image)

root.title('表情包')
tk_image_widget=tk.Label(root,image=tk_image)
tk_image_widget.pack()

def keyboardEventResp(event:tk.Event):
    key=event.keysym
    if key=='s':
        image.save(dir_path+f'/{img_name}.{img_type}')
        root.destroy()
    elif key=='n':
        root.destroy()
    elif key=='q':
        root.destroy()

root.bind('<KeyPress>',keyboardEventResp)

root.mainloop()